# Example Notebook: Estimating Free Water Evaporaiton

In this example we calcuate the free water evpoaration from Lake Hefner, Oklahoma on the 12th of July 1951. The Lake has an area of 9.4 km$^2$ and the estimated mass balance evaporation is **5.81mm**.

On this day the lake has the following aveage values: $T$(2m) = 27.2 [C], $T_s$ = 26.9 [C], $RH$ = 0.69 [], $P$ 97.3 [kPa], $u$(2m) = 5.81 [m/s], $T_{s(pan)}$ = 27.5, $u_{(pan)}$ = 2.79 [m/s]. On this day the lake has the following daily total values: $K_{in}$ = [30.6 9MJ/m$^2$ day], a = 0.052, $L_{in}$ = 34.4 [MJ/m$^2$ day], and $E_{(pan)}$ = 12.4 [mm].

Here we calculate the daily free-water evaporation via the mass-transfer, energy-balance, Bowen ratio, combination, and compare the results with the evaporation given by the lake water balance, which is thought to provide an accurate value for this lake.

## Mass Transfer Methods

The basic mass-transfer relation is

$$ E_0 = K_E u(2m) [e_s - e(2m)] $$

The value of $K_E$ can be calucated from the average wave height ($z_d = 0.0012$) or based on the lake area. Note how correcting for the area of the lake decreases the estimated flux.

In [1]:
# Import numeric python and the plotting library
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Given Information
relativeHumidity = 0.69
windSpeed = 5.81 # [m/s]
T2m = 27.2 # C
Ts = 26.9 # C
zm = 2 # m
press = 97.3 # Estimated air pressure [kPa]
lakeArea = 9.4 # Km^2

# Constants
rhoW = 996 # Density of water [kg/m3]
Rda = 287.04 # Universal gass constant for dry air
vonKarman = 0.4 # Von Karman constant
z0 = 0.0012 # Table 6.4
zd = 0 # Table 6.4

# Vapor pressures in kPa [Equation 6.2]
def eStarFn(inT): return 0.611*np.exp((17.3*inT)/(inT + 237.3))
eAir = eStarFn(T2m)*relativeHumidity
eSurf = eStarFn(Ts)

# Get K_E [Equation 6.11]
rhoA = press*1000/(Rda*(T2m+273.15))*(1-0.378*eAir/press)
K_E = (0.622 * vonKarman**2 * rhoA) / (rhoW * press * (np.log((zm-zd)/(z0)))**2)
K_E = K_E*(1000*60*60*24)# convert to mm/m/day

# Evaporation in mm/hr using Mass Transfer Approach (Eq 6.21)
evapMT = K_E * windSpeed * (eSurf - eAir)
print('Mass transfer total ET (wave height): \t %.1f mm/day'%evapMT)

# KE based on lake area (Equation 6.28)
K_E = 1.26*lakeArea**-0.05
evapMT = K_E * windSpeed * (eSurf - eAir) 
print('Mass transfer total ET (lake Area): \t %.1f mm/day'%evapMT)


Mass transfer total ET (wave height): 	 11.1 mm/day
Mass transfer total ET (lake Area): 	 6.9 mm/day


## The Energy Balance Method

The energy balance can be used to esimate the evaporaiton as

$$  E_0 = \frac{K+L-H}{\rho_w \lambda_v} $$

For the energy balance we need $K$, $L$, and $H$. Once these are solved for we plug into the above equation.

In [2]:
# More Given Information
K_in = 30.6
L_in = 34.4
albedo = 0.052

# More Constants
emissivity_water = 1-albedo
stefanBoltzman = 4.9*10**-9
lambdaV = 2.50 - (2.36*10**-3*Ts) # (equation 6.14)

# Sortwave energy (eq. 6B1.1)
K = K_in*(1-albedo)

# Longwave energy (eq. 6B1.4 and 6B1.6)
L_out = emissivity_water*stefanBoltzman*(Ts+273.15)**4 + (1-emissivity_water)*L_in
L = L_in - L_out

# Sensible heat (equation 6.16 and 6.17)
K_H =  1.95*10**-4/ (np.log((zm-zd)/(z0)))**2
K_H = K_H*(60*60*24)# convert to MJ/m/day
H = K_H * windSpeed * (Ts - T2m) 

# Solve for E_0
evapEnergyBalance = (K + L - H)/(rhoW*lambdaV) *1000
print('Energy balance total ET: \t %.1f mm/day'%evapEnergyBalance)


Energy balance total ET: 	 10.1 mm/day


## The Energy Balance Method with Bowen Ratio

The Bowen Ratio is defined as the ratio of sensible to to latent heat as

$$ B = \frac{H}{LE} = \gamma\frac{[T_s - T(2m)]}{[e_s - e(2m)]} $$

where $\gamma$ ist he psycrhrometric constant. Note the **Evaporative Fraction** is the fraciton of incomming energy that goes into evaporation and is given by

$$ EF = \frac{LE}{H + LE} = \frac{1}{B+1} $$

The energy balance can be used with the bowen ratio to esimate the evaporaiton as

$$  E_0 = \frac{K+L}{\rho_w \lambda_v (1+B)} $$

For the energy balance we need $K$, $L$, and $B$. Once these are solved for we plug into the above equation.

In [3]:
# Calcualte psychrometric constant (equation 6.21)
c_a = 1*10**-3
gamma = (c_a*press)/(0.622*lambdaV)

# Bowen Ratio (eq 6.32)
B = gamma*(Ts-T2m)/(eSurf-eAir)

# Energy Balance Flux
evapBowen = (K + L)/(rhoW*lambdaV*(1+B)) *1000
print('Energy balance with Bowen Ratio total ET: \t %.1f mm/day'%evapBowen)


Energy balance with Bowen Ratio total ET: 	 10.1 mm/day


## Combination

To use the combination equation be need the slope of the vapor pressure curve ($\Delta$). With this we can calculate the evaporation with the combination equation:

$$ E_0 = \frac{\Delta(K+L) + \gamma \rho_w K_E \lambda_v u(2m) [e^*(2m) - e(2m)]}{\rho_w \lambda_v (\Delta + \gamma)} $$

The value of $\Delta$ is calcualated with equation 6.4 and then the combination equation is solved

In [4]:
# Slope of vapor pressure curve
Delta = 2508.3/((T2m+237.3)**2) * np.exp((17.3*T2m)/(T2m+237.3))

# Combinatio Equation (Note that K_E must be adjusted to the correct units)
evapCombo = (Delta*(K + L) + gamma*rhoW*K_E*10**-3*windSpeed*(eStarFn(T2m) - eAir))/(
    (rhoW*lambdaV*(Delta + gamma))) *1000
print('Energy balance total ET: \t %.1f mm/day'%evapCombo)

Energy balance total ET: 	 8.3 mm/day


## Review
Note the range of values here. The observed mass balance estimate was 5.8mm, while the other estimates ranged from 6.9 to 11.1mm. 
